# 6章　データの読み込み、書き出しとファイル形式

In [111]:
import pandas as pd
import sys
import numpy as np
import csv
import json
from lxml import objectify
from io import StringIO
import requests
import sqlite3
import sqlalchemy as sqla

## 6.1 テキスト形式のデータの読み書き

In [2]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [5]:
df = pd.read_csv("ex1.csv")

In [6]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_table("ex1.csv", sep=",") # pd.read_tableは廃止された

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
!cat ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [10]:
pd.read_csv("ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
pd.read_csv("ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [13]:
names = ["a", "b", "c", "d", "message"]

In [14]:
pd.read_csv("ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [16]:
###
pd.read_csv("ex2.csv", names=names, index_col=names[4])

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [17]:
!cat csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [18]:
parsed = pd.read_csv("csv_mindex.csv", index_col=["key1", "key2"])

In [19]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [20]:
list(open("ex3.txt"))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [21]:
result = pd.read_table("ex3.txt", sep="\s+")

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [22]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [23]:
!cat ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [24]:
pd.read_csv("ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [25]:
###
pd.read_csv("ex4.csv")

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [26]:
!cat ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [27]:
result = pd.read_csv("ex5.csv")

In [28]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [29]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [30]:
result = pd.read_csv("ex5.csv", na_values=["NULL"])

In [31]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [32]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}

In [33]:
pd.read_csv("ex5.csv", na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 6.1.1 テキストファイルを少しずつ読み込む

In [34]:
pd.options.display.max_rows = 10

In [35]:
result = pd.read_csv("ex6.csv")

In [36]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [37]:
pd.read_csv("ex6.csv",  nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [55]:
chunker = pd.read_csv("ex6.csv", chunksize=1000)

In [56]:
chunker

In [60]:
chunker = pd.read_csv("ex6.csv", chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [61]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [63]:
###
chunker = pd.read_csv("ex6.csv", chunksize=1000)
chunker.get_chunk(5)
# chunk引数を適用したTextFileReaderオブジェクトは使う都度作らないとダメっぽい

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### 6.1.2 テキスト形式でのデータの書き出し

In [64]:
data = pd.read_csv("ex5.csv")

In [65]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [66]:
data.to_csv("out.csv")

In [67]:
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [69]:
data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [71]:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [72]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [73]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [74]:
dates = pd.date_range("1/1/2000", periods=7)

In [76]:
ts = pd.Series(np.arange(7), index=dates)

In [77]:
ts.to_csv("tseries.csv")

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [78]:
!cat tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### 6.1.3 区切り文字で区切られた形式を操作する

In [79]:
!cat ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [100]:
f = open("ex7.csv")

In [93]:
reader = csv.reader(f)

In [94]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [95]:
with open("ex7.csv") as f:
    lines = list(csv.reader(f))

In [96]:
header, values = lines[0], lines[1:]

In [97]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [98]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [101]:
class my_dialect(csv.Dialect):
    lineterminater = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f, dialect=my_dialect)

TypeError: lineterminator must be set

In [102]:
reader = csv.reader(f, delimiter="|")

In [103]:
with open("mydata.csv", "w") as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4", "5", "6"))
    writer.writerow(("7", "8", "9"))

TypeError: lineterminator must be set

### 6.1.4 JSONデータ

In [7]:
obj = """
{"name": "Wes",
  "places_loved": ["United States", "Spain", "Germany"],
   "pet": null,
   "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
                      {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [8]:
result = json.loads(obj)

In [9]:
result

{'name': 'Wes',
 'places_loved': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [10]:
asjson = json.dumps(result)

In [11]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])

In [12]:
siblings

,name,age
0,Scott,30
1,Katie,38


In [13]:
!cat example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [14]:
data = pd.read_json("example.json")

In [15]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [16]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [17]:
print(data.to_json(orient="records"))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### 6.1.5 XMLとHTML：ウェブスクレイピング

In [19]:
tables = pd.read_html("fdic_failed_bank_list.html")

In [20]:
len(tables)

1

In [27]:
failures = tables[0]

In [28]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [30]:
###
tables.head()

AttributeError: 'list' object has no attribute 'head'

In [31]:
close_timestamps = pd.to_datetime(failures["Closing Date"])

In [33]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

### 6.1.5.1 lxml.objectifyを使ったXMLの読み込み

In [37]:
path = "Performance_MNR.xml"
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [40]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ", "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [41]:
perf = pd.DataFrame(data)

In [42]:
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [44]:
tag = '<a href="http:/www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [45]:
root

<Element a at 0x123bb77c8>

In [46]:
root.get("href")

'http:/www.google.com'

In [47]:
root.text

'Google'

## 6.2 バイナリデータ形式

In [60]:
frame = pd.read_csv("ex1.csv")

In [61]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [62]:
frame.to_pickle("frame_pickle")

In [63]:
pd.read_pickle("frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 6.2.1 HDF5形式の使用

In [64]:
frame = pd.DataFrame({"a": np.random.randn(100)})

In [65]:
store = pd.HDFStore("mydata.h5")

In [67]:
store["obj1"] = frame

In [68]:
store["obj1_col"] = frame["a"]

In [69]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [70]:
store["obj1"]

,a
0,1.337161
1,-0.747997
2,0.907892
3,0.724153
4,-1.497291
5,-0.669719
6,0.012117
7,0.068369
8,0.738926
9,1.592272


In [72]:
store.put("obj2", frame, format="table")

In [73]:
store.select("obj2", where=["index >= 10 and index <= 15"])

,a
10,-0.675248
11,-1.392926
12,-0.823378
13,-0.560976
14,-0.105681
15,0.587728


In [74]:
store.close()

In [76]:
frame.to_hdf("mydata.h5", "obj3", format="table")

In [77]:
pd.read_hdf("mydata.h5", "obj3", where=["index < 5"])

,a
0,1.337161
1,-0.747997
2,0.907892
3,0.724153
4,-1.497291


### 6.2.2 Microsoft Excelファイルの読み込み

In [78]:
xlsx = pd.ExcelFile("ex1.xlsx")

In [79]:
###
xlsx

In [80]:
pd.read_excel(xlsx, "Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [81]:
frame = pd.read_excel("ex1.xlsx", "Sheet1")

In [82]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [83]:
writer = pd.ExcelWriter("ex2.xlsx")

In [84]:
frame.to_excel(writer, "Sheet1")

In [85]:
writer.save()

In [86]:
frame.to_excel("ex2.xlsx")

## 6.3 Web APIを用いたデータの取得

In [88]:
url = "https://api.github.com/repos/pandas-dev/pandas/issues"

In [89]:
resp = requests.get(url)

In [90]:
resp

<Response [200]>

In [91]:
data = resp.json()

In [92]:
data[0]["title"]

'TST: coverage config ignore e.g. _version.py'

In [93]:
issues = pd.DataFrame(data, columns=["number", "title", "labels", "state"])

In [94]:
issues

,number,title,labels,state
0,26877,TST: coverage config ignore e.g. _version.py,[],open
1,26876,Enhancement Add max_level param to json_normalize,[],open
2,26875,BUG: from_dict ignored order of OrderedDict (#...,"[{'id': 1049312478, 'node_id': 'MDU6TGFiZWwxMD...",open
3,26873,STY: rename `filter` kwarg in `replace` methods,[],open
4,26872,Add docstring to the insertion method & add em...,[],open
5,26871,Add type hints for (DataFrame|Series)._data,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open
6,26870,Add typing annotation to IntervalIndex.interse...,[],open
7,26869,Refactor pandas._typing And Add Contributing G...,"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open
8,26868,DOC: Reference to type 'pd_table' in IO Tools ...,[],open
9,26867,Annotate DataFrame (Part 1),"[{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...",open


## 6.4 データベースからのデータの取得

In [96]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,                  d INTEGER
 );"""

In [97]:
con = sqlite3.connect("mydata.sqlite")

In [98]:
con.execute(query)

In [99]:
con.commit()

In [100]:
data = [("Atlanta", "Gergia", 1.25, 6),
              ("Tallahassee", "Floroda", 2.6, 3),
              ("Sacramento", "California", 1.7, 5)]

In [101]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [103]:
con.executemany(stmt, data)

In [105]:
con.commit()

In [106]:
cursor = con.execute("select * from test")

In [107]:
rows = cursor.fetchall()

In [108]:
rows

[('Atlanta', 'Gergia', 1.25, 6),
 ('Tallahassee', 'Floroda', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [109]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [110]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Gergia,1.25,6
1,Tallahassee,Floroda,2.60,3
2,Sacramento,California,1.70,5


In [113]:
db = sqla.create_engine("sqlite:///mydata.sqlite")

In [114]:
pd.read_sql("select * from test", db)

,a,b,c,d
0,Atlanta,Gergia,1.25,6
1,Tallahassee,Floroda,2.60,3
2,Sacramento,California,1.70,5
